In [1]:
import pandas as pd
import glob
import os
from pprint import pprint

In [2]:
#Get all CSV files in the folder of GA
all_files = glob.glob(r"C:\Huy Phan\College\VoterTurnout\data\PA\2016\*.csv")

# Files that contain both 'precinct' and 'general' in the filename
general_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'general'])
]


# Files that contain both 'precinct' and 'primary' in the filename
primary_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'primary', 'president'])
]


In [3]:
print("General files:")
for f in general_files:
    print(f)


General files:


In [4]:
print("\nPrimary files:")
for f in primary_files:
    print(f)


Primary files:


In [98]:
# Process primary files
primary_df_list = []

for file in primary_files:
    try:
        df = pd.read_csv(file)

        # Drop rows where 'precinct' is NaN
        if 'precinct' in df.columns:
            df = df[df["precinct"].notna()]
        
        df = df.drop_duplicates()

        primary_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
pri_combined_df = pd.concat(primary_df_list, ignore_index=True)
# Create a new column "precinct" by combining county and precinct
pri_combined_df["precinct"] = pri_combined_df["county"].astype(str) + pri_combined_df["precinct_code"].astype(str)
pri_combined_df = pri_combined_df.drop(columns=["party"])
pri_combined_df = pri_combined_df.rename(columns={"office": "party"})
pri_combined_df


,county,precinct_code,precinct,party,district,candidate,votes
0,King,3562,King3562,Democratic Party,NaN,Bernie Sanders,24
1,King,3562,King3562,Democratic Party,NaN,Hillary Clinton,117
2,King,3562,King3562,Democratic Party,NaN,Registered Voters,503
3,King,3562,King3562,Democratic Party,NaN,Write-In,0
4,King,3562,King3562,Republican Party,NaN,Ben Carson,1
...,...,...,...,...,...,...,...
52753,Yakima,5101,Yakima5101,President Republican Party,NaN,Donald J. Trump,72
52754,Yakima,5202,Yakima5202,President Republican Party,NaN,Ben Carson,0
52755,Yakima,5202,Yakima5202,President Republican Party,NaN,Ted Cruz,0
52756,Yakima,5202,Yakima5202,President Republican Party,NaN,John R. Kasich,0


In [99]:
pri_combined_df["party"].value_counts(dropna=False)

party
President Republican Party    18392
Republican Party              15102
Democratic Party              10068
President Democratic Party     9196
Name: count, dtype: int64

In [100]:
# Select only the relevant columns
primary_data = pri_combined_df[["precinct", "party", "candidate", "votes"]]
primary_data.loc[:, "party"] = primary_data["party"].replace({
    "President Republican Party": "REP",
    "President Democratic Party": "DEM",
    "Democratic Party": "DEM",
    "Republican Party": "REP"
})
primary_data = primary_data[primary_data["party"].isin(["DEM", "REP"])] # Analyzing only republican and democratic

primary_data

,precinct,party,candidate,votes
0,King3562,DEM,Bernie Sanders,24
1,King3562,DEM,Hillary Clinton,117
2,King3562,DEM,Registered Voters,503
3,King3562,DEM,Write-In,0
4,King3562,REP,Ben Carson,1
...,...,...,...,...
52753,Yakima5101,REP,Donald J. Trump,72
52754,Yakima5202,REP,Ben Carson,0
52755,Yakima5202,REP,Ted Cruz,0
52756,Yakima5202,REP,John R. Kasich,0


In [101]:
primary_data["party"].value_counts(dropna=False)

party
REP    33494
DEM    19264
Name: count, dtype: int64

In [102]:
#Remove Write-in and Registered Voters
primary_data = primary_data[~primary_data["candidate"].isin(["Registered Voters", "Write-In"])] 
primary_data["candidate"].value_counts(dropna=False)

candidate
Bernie Sanders     7115
Hillary Clinton    7115
Ben Carson         7115
Donald J. Trump    7115
John R. Kasich     7115
Ted Cruz           7115
Name: count, dtype: int64

In [103]:
primary_data["candidate"].unique()
candidate_party_map = (
    primary_data.dropna(subset=["candidate", "party"])
                .set_index("candidate")["party"]
                .to_dict()
)
print(candidate_party_map)

{'Bernie Sanders': 'DEM', 'Hillary Clinton': 'DEM', 'Ben Carson': 'REP', 'Donald J. Trump': 'REP', 'John R. Kasich': 'REP', 'Ted Cruz': 'REP'}


In [104]:
primary_data.loc[:,"candidate_column"] = (
    "pri_" +
    primary_data["party"].str.lower() + "_" +
    primary_data["candidate"].str.split().str[-1].str.upper()
)

# pivot the table
primary_result = primary_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

primary_result

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_24240\2782897409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data.loc[:,"candidate_column"] = (


candidate_column,precinct,pri_dem_CLINTON,pri_dem_SANDERS,pri_rep_CARSON,pri_rep_CRUZ,pri_rep_KASICH,pri_rep_TRUMP
0,Adams111,11,6,2,6,2,45
1,Adams112,14,10,2,6,1,41
2,Adams113,17,11,8,8,2,59
3,Adams114,16,19,1,10,13,44
4,Adams115,18,11,4,5,11,53
...,...,...,...,...,...,...,...
7110,Yakima5020,26,11,6,15,12,100
7111,Yakima5101,19,26,0,6,5,72
7112,Yakima5202,0,0,0,0,0,0
7113,Yakima701,26,25,10,3,2,22


In [105]:
# Process general files
gen_df_list = []

for file in general_files:
    try:
        df = pd.read_csv(file)

        # Select only president
        if 'office' in df.columns:
            df = df[df["office"] == "President" ]

        gen_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
gen_combined_df = pd.concat(gen_df_list, ignore_index=True)
# Create a new column "precinct" by combining county and precinct
gen_combined_df["precinct"] = gen_combined_df["county"].astype(str) + gen_combined_df["precinct_code"].astype(str)
gen_combined_df


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_24240\1650111261.py:6: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


,county,precinct_code,precinct,office,district,party,candidate,votes
0,King,3562,King3562,President,NaN,Constitution,Darrell L. Castle & Scott N. Bradley,0
1,King,3562,King3562,President,NaN,Democratic,Hillary Clinton & Tim Kaine,266
2,King,3562,King3562,President,NaN,Green,Jill Stein & Ajamu Baraka,1
3,King,3562,King3562,President,NaN,Libertarian,Gary Johnson & Bill Weld,14
4,King,3562,King3562,President,NaN,NaN,Registered Voters,519
...,...,...,...,...,...,...,...,...
54911,Yakima,5202,Yakima5202,President,NaN,Socialist Workers,Alyson Kennedy / Osborne Hart,0
54912,Yakima,5202,Yakima5202,President,NaN,Socialism & Liberation,Gloria Estela La Riva / Eugene Puryear,0
54913,Yakima,5202,Yakima5202,President,NaN,Green,Jill Stein / Ajamu Baraka,0
54914,Yakima,5202,Yakima5202,President,NaN,Constitution,Darrell L. Castle / Scott N. Bradley,0


In [106]:
general_data = gen_combined_df[["precinct", "party", "candidate", "votes"]]
general_data["party"].value_counts(dropna=False)


party
Constitution              7126
Democratic                7126
Green                     7126
Libertarian               7126
Republican                7126
Socialism & Liberation    7126
Socialist Workers         7126
NaN                       5034
Name: count, dtype: int64

In [107]:
general_data["candidate"].value_counts(dropna=False)

candidate
Hillary Clinton / Tim Kaine               4609
Donald J. Trump / Michael R. Pence        4609
Alyson Kennedy / Osborne Hart             4609
Gloria Estela La Riva / Eugene Puryear    4609
Jill Stein / Ajamu Baraka                 4609
Darrell L. Castle / Scott N. Bradley      4609
Gary Johnson / Bill Weld                  4609
Darrell L. Castle & Scott N. Bradley      2517
Hillary Clinton & Tim Kaine               2517
Jill Stein & Ajamu Baraka                 2517
Gary Johnson & Bill Weld                  2517
Registered Voters                         2517
Write-In                                  2517
Donald J. Trump & Michael R. Pence        2517
Gloria Estela La Riva & Eugene Puryear    2517
Alyson Kennedy & Osborne Hart             2517
Name: count, dtype: int64

In [108]:
general_data["candidate"] = general_data["candidate"].str.extract(r"^([^/&]+)").iloc[:, 0].str.strip()
general_data = general_data[~general_data["candidate"].isin(["Registered Voters", "Write-In"])] 
general_data["candidate"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_24240\1105235521.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate"] = general_data["candidate"].str.extract(r"^([^/&]+)").iloc[:, 0].str.strip()


candidate
Darrell L. Castle        7126
Hillary Clinton          7126
Jill Stein               7126
Gary Johnson             7126
Donald J. Trump          7126
Gloria Estela La Riva    7126
Alyson Kennedy           7126
Name: count, dtype: int64

In [114]:
# general_data = general_data[~general_data["candidate"].isin(["YES", "NO"])]
# general_data["party"] = general_data.apply(
#     lambda row: candidate_party_map.get(row["candidate"], row["party"]) if pd.isna(row["party"]) else row["party"],
#     axis=1
# )
general_data.loc[:, "party"] = general_data["party"].replace({
    "Democratic": "DEM",
    "Republican": "REP"
})
general_data = general_data[general_data["party"].isin(["DEM", "REP"])] # Analyzing only republican and democratic
general_data

,precinct,party,candidate,votes,candidate_column
1,King3562,DEM,Hillary Clinton,266,gen_dem_CLINTON
6,King3562,REP,Donald J. Trump,185,gen_rep_TRUMP
10,King2381,DEM,Hillary Clinton,334,gen_dem_CLINTON
15,King2381,REP,Donald J. Trump,215,gen_rep_TRUMP
19,King3400,DEM,Hillary Clinton,235,gen_dem_CLINTON
...,...,...,...,...,...
54896,Yakima5020,REP,Donald J. Trump,240,gen_rep_TRUMP
54902,Yakima5101,DEM,Hillary Clinton,91,gen_dem_CLINTON
54903,Yakima5101,REP,Donald J. Trump,176,gen_rep_TRUMP
54909,Yakima5202,DEM,Hillary Clinton,0,gen_dem_CLINTON


In [115]:
general_data["candidate_column"] = (
    "gen_" +
    general_data["party"].str.lower() + "_" +
    general_data["candidate"].str.split().str[-1].str.upper()
)

# pivot the table
general_result = general_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

general_result

candidate_column,precinct,gen_dem_CLINTON,gen_rep_TRUMP
0,Adams111,24,109
1,Adams112,26,96
2,Adams113,56,117
3,Adams114,54,113
4,Adams115,48,123
...,...,...,...
7121,Yakima5020,95,240
7122,Yakima5101,91,176
7123,Yakima5202,0,0
7124,Yakima701,97,53


In [116]:
combined = pd.merge(primary_result, general_result, on="precinct", how="inner")
combined["rep_primary_total"] = combined.filter(like="pri_rep_").sum(axis=1)
combined["dem_primary_total"] = combined.filter(like="pri_dem_").sum(axis=1)
combined["general_total"] = combined.filter(like="gen_").sum(axis=1)
combined

candidate_column,precinct,pri_dem_CLINTON,pri_dem_SANDERS,pri_rep_CARSON,pri_rep_CRUZ,pri_rep_KASICH,pri_rep_TRUMP,gen_dem_CLINTON,gen_rep_TRUMP,rep_primary_total,dem_primary_total,general_total
0,Adams111,11,6,2,6,2,45,24,109,55,17,133
1,Adams112,14,10,2,6,1,41,26,96,50,24,122
2,Adams113,17,11,8,8,2,59,56,117,77,28,173
3,Adams114,16,19,1,10,13,44,54,113,68,35,167
4,Adams115,18,11,4,5,11,53,48,123,73,29,171
...,...,...,...,...,...,...,...,...,...,...,...,...
7110,Yakima5020,26,11,6,15,12,100,95,240,133,37,335
7111,Yakima5101,19,26,0,6,5,72,91,176,83,45,267
7112,Yakima5202,0,0,0,0,0,0,0,0,0,0,0
7113,Yakima701,26,25,10,3,2,22,97,53,37,51,150


In [112]:
combined.to_csv("WA.csv", index=False)
